### TODO

Derive the relationship between Modified Duration and Macaulay Duration.  

Note:  
**Macaulay Duration = $-\frac{1}{P}\frac{dP}{dr}$**  
Where r is the continously compounded yield.  
**Modified Duration = $\frac{1}{1+y/f}D$**  
where y is the yield and f is the coumpunding frequency of the yield in question.

**Instantaneous Rate**

Rate of return over small period $dt$  
**$r(t)=\operatorname{Lt}_{dt\rightarrow0} \frac{1}{dt}\frac{B(t+dt)-B(t)}{B(t)}=\frac{B'(t)}{B(t)}$**
Integrating both sides  
$\int_0^tr(\tau)d\tau=\int_0^t\frac{B'(\tau)}{B(\tau)}d\tau$  
$\int_0^tr(\tau)d\tau=\left[log(B(\tau))\right]_0^t$    
$\int_0^tr(\tau)d\tau=log(\frac{B(t)}{B(0)})$    
Present Value using instaneous rate $r(\tau)$  
$B_t=B_0e^{\int_0^tr(\tau)d\tau}$  

**Zero rate Vs Instantaneous rate relationship**  
$e^{\int_0^tr(\tau)d\tau}=e^{tr(0,t)}$  
#### $r(0,t)=\frac{1}{t}\int_0^tr(\tau)d\tau$  
${\int_0^tr(\tau)d\tau}=tr(0,t)$  
Differentiating both sides you get(Left side is from integral calc lemma)
$r(t)(t)'-r(0)(0)'=r(0,t)+t\frac{d}{dt}r(0,t)$  
#### $r(t)=r(0,t)+t\frac{d}{dt}r(0,t)$  

If rate is contant over a period is it Zero rate or Instantaneous rate?  
$r(0,t)=\frac{1}{t}\int_0^tr(\tau)d\tau$  
Let $r(\tau)$ be constant $r$ over $0$ to $t$  
$r(0,t)=\frac{r}{t}\int_0^td\tau$  
$r(0,t)=r$  
Thus $r(0,t)=r(\tau)=r \quad\forall\quad \tau\epsilon [0,t]$     

CDS Spreads
--

If yearly probability of default is 2% conditional that no earlier default happened. 

$S =$1-0.02=0.98   
$S=e^{-\lambda t}$  
$\lambda = -ln(0.98)$  


In [1]:
from math import log
print("Hazard Rate(continuos compounding) for 2%-annual-default-rate(non compounding) is {:.2%}".format(-log(0.98)))


Hazard Rate(continuos compounding) for 2%-annual-default-rate(non compounding) is 2.02%


Bond Prices to CDS Premium to Default Probability
--

In [60]:
import numpy as np
import math

risk_free_rate=0.05
bond_ytm=0.07
cpn=0.06
mat=5
rec=0.4

print(risk_free_rate)

CpnCashFlows=np.full((mat*2),cpn*50)
PrinCashFlows=np.full((mat*2),0)
PrinCashFlows[-1]=100
DiscountFactors=[math.exp(-i*bond_ytm/2) for i in range(1,mat*2+1)]
print("Coupon Cashflows==")
print(CpnCashFlows)
print("Principal Cashflows==")
print(PrinCashFlows)
print("Discount Factors==") 
print(DiscountFactors)
print("Discounted Principal==")
print(PrinCashFlows*DiscountFactors)
print("Discounted Coupons==")
print(CpnCashFlows*DiscountFactors)
print("PV of FV=" + str(np.sum(CpnCashFlows*DiscountFactors)))
BondValue=np.sum(PrinCashFlows*DiscountFactors)+ np.sum(CpnCashFlows*DiscountFactors)
print("Bond Price=" + str(BondValue))
RiskFreeDiscountFactors=[math.exp(-i*risk_free_rate/2) for i in range(1,mat*2+1)]
RiskFreeBondPrice=np.sum(PrinCashFlows*RiskFreeDiscountFactors)+ np.sum(CpnCashFlows*RiskFreeDiscountFactors)
print("Risk Free Bond Price= " + str(RiskFreeBondPrice))
DefaultPV=RiskFreeBondPrice-BondValue
print("Default PV=" + str(DefaultPV))
CDS_Premium_DFs=[1,*RiskFreeDiscountFactors[:-1]]
print("DFs for premium payments=")
print(CDS_Premium_DFs)
SemiPremPayments=DefaultPV/sum(CDS_Premium_DFs)
print("Semi Annual CDS Premium Payment in Dollars=" + str(SemiPremPayments))
CDS_Spread_Annual=SemiPremPayments/100.*2.
print("Annual CDS Premium Payment in Percentage={:.5}bps".format(CDS_Spread_Annual*10000))
Annual_Def_prob=CDS_Spread_Annual/(1-rec)
print("Annual Default Rate or Constant hazard Rate={:.2%} per annum".format(Annual_Def_prob))


0.05
Coupon Cashflows==
[3. 3. 3. 3. 3. 3. 3. 3. 3. 3.]
Principal Cashflows==
[  0   0   0   0   0   0   0   0   0 100]
Discount Factors==
[0.9656054162575665, 0.9323938199059483, 0.9003245225862656, 0.8693582353988059, 0.8394570207692074, 0.8105842459701871, 0.7827045382418681, 0.7557837414557255, 0.7297888742690568, 0.7046880897187134]
Discounted Principal==
[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.         70.46880897]
Discounted Coupons==
[2.89681625 2.79718146 2.70097357 2.60807471 2.51837106 2.43175274
 2.34811361 2.26735122 2.18936662 2.11406427]
PV of FV=24.872065513720035
Bond Price=95.34087448559137
Risk Free Bond Price= 104.09356799388405
Default PV=8.75269350829268
DFs for premium payments=
[1, 0.9753099120283326, 0.951229424500714, 0.9277434863285529, 0.9048374180359595, 0.8824969025845955, 0.8607079764250578, 0.8394570207692074, 0.8187307530779818, 0.7985162187593771]
Semi Annual CDS Premium Payment in Dollars=0.97696897

In [61]:
#Calculating Default probabilities from Asset Swap Spreads
Bond_Asset_Swap_Spread=150 #Basis points
CF_Extra_Payments=np.full((mat*2),150/100/2.)
print("CF_Extra_Payments={}".format(CF_Extra_Payments))
DefPV=sum(CF_Extra_Payments*RiskFreeDiscountFactors)
print(DefPV)
Annual_Def_prob=150/(1-rec)/10000
print("Annual Default Rate or Constant hazard Rate={:.2%} per annum".format(Annual_Def_prob))

CF_Extra_Payments=[0.75 0.75 0.75 0.75 0.75 0.75 0.75 0.75 0.75 0.75]
6.553372421685887
Annual Default Rate or Constant hazard Rate=2.50% per annum


In [79]:
#Misc
Hazard_Rate=0.17/100. #Annual Hazard Rate
Recovery=40/100
Expected_Annual_Spread=Hazard_Rate*(1-Recovery)
print("{:.2f} bps".format(Expected_Annual_Spread*10000))

10.20 bps


In [ ]:
#Misc Merton Formula
E0=3000000
SigmaE=80/100.
DT=10000000
RiskFreeRate=5/100.


$E_0=e^{-r\tau}[N(d_+)V_\tau - N(d_-)D_\tau]$  
$E_0$ is Company Equity Value today.  
$V_\tau$ is Company Asset Value at $\tau$.  
$D_\tau$ is Company Debt payments at $\tau$.  
$N(d_+)=\frac{ln(\frac{V_\tau}{D_\tau}) + (r-\frac{\sigma^2}{2})\tau}{\sigma\sqrt{\tau}}$  

Alternatively  
$E_0=[N(d_+)V_\tau e^{-r\tau} - N(d_-)D_\tau e^{-r\tau}]$  
$E_0=[N(d_+)V_0 - N(d_-)D_\tau e^{-r\tau}]$   
$N(d_+)=\frac{ln(\frac{V_0}{D_\tau}) + (r-\frac{\sigma^2}{2})\tau}{\sigma\sqrt{\tau}}$  
--
$\qquad$ Not sure why $D_\tau$ does not carry the $e^{-r\tau}$ along. Need to look.  

And we have from *Ito's Lemma*  

$E_0\sigma_e=\frac{\delta E}{\delta V}V_0\sigma_v$  
$E_0\sigma_e=N(d_+)V_0\sigma_v$  


Black Scholes
--

$C_0=e^{-rt}[S_tN(d_+)-KN(d_-)]$  
$d_+=\frac{1}{\sigma \sqrt{t}} (ln(\frac{S_t}{K})+\frac{\sigma^2}{2}t)$  
$d_+=\frac{1}{\sigma \sqrt{t}} (ln(\frac{S}{K})+(r+\frac{\sigma^2}{2})t)$  
$d_-=d_+ - \sigma\sqrt{t}$  
Note that $S_t=e^{rt}S$  

### Gausian Copula

If $Q_i(\tau_i)$ is the probability of default of a Bond $i$ by time $t_i$ in a portfolio.  
$Q_i(\tau_i)$ has the properties of Cumulative Probability Distribution for Random Variable $t_i$.  
$x_i=N^{-1}(Q_i(\tau))$ is a Standard Normal Variable $t_i$.  
Now all $x_i$ are Standard Normal Variables, whose joint distribution $Z=[x_1,x_2...x_n]$ will also be Standard Normal.  

For any joint distributions we need Correlations for them to be Correlated Standard Normal Variables. This is because we are trying get the effect of relationships between various parts of the portfolio or the credit risky assets in a portfolio.

One way to simulate this is by feeding in pairwise correlations. Generate correlated random variables $x_1,x_2...x_n$. This will give you $N(x_i)=Q_i(t_i)$. Now that you have probability of default for $i$ you can get the expected time to default from the Bond/Credit $i$'s inherent properties(Say from Ratings tables of historical cumulative default rates.)

## Using Implied Correlation(Common Correlation)

$x_i=a_iF + \sqrt{1-a_i^2}Z_i$  
$\qquad$Where  
$\qquad F$ is a common factor effecting all entities in a portfolio, call it a macro economic factor.  
$\qquad Z_i$ is the marginal or idiosyncratic effect.  
$\qquad a_i$ is a constant parameter and the correlation between $x_i$ and $x_j$ is $a_ia_j$. Thus $a_i$ can approximated as the entity $i$'s equity correlation with a well diversified market index.  

A Bond will default in time $t_i$ if $N(x_i)<Q_i(t_i)$.  
$x_i<N^{-1}(Q_i(t_i))$  
$a_iF + \sqrt{1-a_i^2}Z_i<N^{-1}(Q_i(t_i))$  
$ Z_i<\frac{N^{-1}(Q_i(t_i))-a_iF}{\sqrt{1-a_i^2}}$  
$ Q_i(\tau|F)=N\left(\frac{N^{-1}(Q_i(t_i))-a_iF}{\sqrt{1-a_i^2}}\right)$  

Usually in Gaussian Copula if the correlation between all assets is a common $\rho$ then  
$Q_i(\tau|F)=N\left(\frac{N^{-1}(Q_i(t_i))-\sqrt\rho F}{\sqrt{1-\rho}}\right)$  

This is the percentage of defaults by time $\tau$ as a function of $F$.  

As $F$ is a standard normal variable.   
We are X% certain that $F$'s value will be greater than $N^{-1}(1-X)= -N^{-1}(X)$.  
When we make sure of the above we would be X% certain that percentage of defaults would be less than $Q_i(\tau|F)$.  
So if we do $F=-N^{-1}(X)$  

$V(X,T)=N\left(\frac{N^{-1}(Q_i(t_i))+\sqrt\rho N^{-1}(X)}{\sqrt{1-\rho}}\right)$  


$N^{-1}(F)$ can be viewed as a confidence level like in VAR with which you want your percentage of defaults to be evaluated thus to calculate a 99% VAR for a credit portfolio we use the below to calculate percentage of defaults.

$V(X,T)=N\left(\frac{N^{-1}(Q_i(t_i))+\sqrt\rho N^{-1}(X)}{\sqrt{1-\rho}}\right)$  

Example:

Average probability if defaults for 1 year = 2%.  
Common(Implied) Correlations of all the assets is 0.1.  
Bank portfolio is worth 100$.  
Average recovery is 60%.  
What is the expected losses for year 1.  

$V(99\%,1yr)=N\left(\frac{N^{-1}(0.02)+\sqrt{0.1} N^{-1}(0.99)}{\sqrt{1-0.1}}\right)$  


In [97]:
import numpy as np
from scipy.stats import norm
Q=0.02
Rho=0.1
Confidence=0.99
Portfolio=100000000
Recovery=60/100.
print(norm.cdf(0))
print(norm.ppf(0.5))

DefaultPercentage=norm.cdf((norm.ppf(Q) + Rho**0.5*norm.ppf(Confidence))/(1-Rho)**0.5)
print("Percentage of defaults ={:4f}".format(DefaultPercentage))

LossGivenDefault=Portfolio*(1-Recovery)
ExpectedLoss=LossGivenDefault*DefaultPercentage
print("99% confidence expected loss on the portfolio is {:,.0f} dollars".format(ExpectedLoss))

0.5
0.0
Percentage of defaults =0.082357
99% confidence expected loss on the portfolio is 3,294,271 dollars


For a kth to Default or for a tranche.
--

Given that  
#### $Q(t|F)=N\left(\frac{N^{-1}(Q(t))-\sqrt\rho F}{\sqrt{1-\rho}}\right)$  

By binomial distribution probability of k defaults can be derived as below:-  
#### $P(k ,t|F)=\frac{n!}{k!(n-k)!}Q(t|F)^k\left[1-Q(t|F)^{n-k}\right]$  

For tranches with attachement $\alpha_a$ and detachment $\alpha_d$ points you can derive the $k_a$ to $k_d$ defaults that would be affecting the tranche payouts on default as below:-

if $m(x)$ is the smallest integer greater than x then

#### $k_a=m\left(\frac{n*\alpha_a}{1-R}\right)$ and $k_d=m\left(\frac{n*\alpha_d}{1-R}\right)$

if $k_a\leq k \leq k_d$ then tranche notional would be(Original tranche notional being \$1)

## $\qquad\frac{\alpha_d- \frac{k(1-R)}{n}}{\alpha_d-\alpha_a}$



### This is very theoretical, remember that in case of default loss is taken by the lowest tranche and the recovered amount is paid back to the Super Senior tranches.

#### In reality if recovery is 40$\%$ then a tranche 60-100 should never take losses.  

#### Tranche Implied correlations have smile. Thus we use base correlation which is more of skew and manageable. Any non standard tranche is not evaluated by interpolating the base correlation, but rather by interpolating the Expected Loss on the standard points of Base tranches $0-i\%$, $0-j\%$ and $0-k\%$ tranches to evaluate Expected Losses on  $0-x\%$ and $0-y\%$. This gives no arbitrage results on Expected Losses. As Expected Losses are more or less linear as we go up the capital structure, but Base Correlations are not linear as we go up the capital structure. Infact the are very convex.



